# Integrating the Sagemaker endpoint into our movie recommender web application

In its current form the K-NN model which we deployed in our previous lab to get recommended movies for a user works nicely from a Jupyter notebook. However it is hard to integrate into a business application. It requires a user embedding vector as input and returns a JSON output which is specific to our model (e.g. movie distances) and contains information which is not relevant to our application.

In the next step we will write a simple lambda function which takes a user id as input and returns a simple list of recommended movies in JSON format. The lambda function will be responsible to transform the user id into a user embedding, invoke the sagemaker endpoint and return a simplified response containing only movie ids to the user. See following figure a simple architecture:

<font color=red> TODO paint nice archtiecture diagram </font>

Adding this integration layer creates a simple REST-API based contract between our business application and our machine learning backend functionality, which has following benefits:

- It allows us to change the deployed machine learning model in the backend without requiring any code changes to our business application
- Allows to easily split the work/responsibilities into separate teams (e.g. bussiness app development and data science team)

# Developing and deploying the lambda function

For deployment of the lambda function we will use the [serverless framework](https://serverless.com/). Let's start by installing the serverless framework and required dependencies.

In [ ]:
!npm install -g serverless serverless-iam-roles-per-function

All required files for deployment are available in the integrationLambda folder. 
The file [serverless.yml](./integrationLambda/serverless.yml) contains all configuration required for deployment, e.g. lambda function properties, permissions or environment variables.
The file [handler.py](./integrationLambda/handler.py) cotnains the code of the lambda function.

<font color=red> Add code explanation for lambda here </font>

Next we need to set a few environment variables, so the lambda knows which sagemaker endpoint to call and where user embeddings are stored.
In serverless.yml update following two properties:

```
EMBEDDINGS_S3_PATH: <insert the path to the embeddings file show at the end of the previous lab notebook>
SAGEMAKER_ENDPOINT_NAME: <insert the name of the sagemaker endpoint also shown at the end of the previous notebook>
```

here is an example:

```
EMBEDDINGS_S3_PATH: s3://sagemaker-assets-jlanger/fm/output/factorization-machines-2019-10-24-19-30-22-215/output/user_embeddings.pickle
SAGEMAKER_ENDPOINT_NAME: knn-2019-10-24-19-35-08-799
```

<font color=red> Update sagemaker notebook to display both properties at the end</font>

Now let's go ahead and deploy the lambda function

In [72]:
!cd integrationLambda; sls deploy

Serverless: Packaging service...
Serverless: Excluding development dependencies...
Serverless: Service files not changed. Skipping deployment...
Service Information
service: recommendationendpointintegration
stage: dev
region: us-east-1
stack: recommendationendpointintegration-dev
resources: 10
api keys:
  None
endpoints:
  GET - https://8c7g5h0jv7.execute-api.us-east-1.amazonaws.com/dev/recommendEndpoint
functions:
  hello: recommendationendpointintegration-dev-hello
layers:
  None
Serverless: Run the "serverless" command to setup monitoring, troubleshooting and testing.


## Validating the endpoint

Now check that the endpoint returns valid results. You can find the url in the output of the last command under endpoints. MAke sure to attach a URL parameter to pass in the requested user_id.
The URL should have following format:

https://\<unique_id>.execute-api.us-east-1.amazonaws.com/dev/recommendEndpoint?user_id=\<user_id>

Here is an example:

https://8c7g5h0jv7.execute-api.us-east-1.amazonaws.com/dev/recommendEndpoint?user_id=3



The request should return a response similar to this (movie ID's can vary):

```
{"movies": [814.0, 1125.0, 653.0, 428.0, 1525.0, 60.0, 652.0, 185.0, 654.0, 86.0, 223.0, 12.0, 1656.0, 191.0, 1130.0, 478.0, 851.0, 137.0, 1467.0, 198.0, 1599.0, 515.0, 1616.0, 1662.0, 187.0, 50.0, 1604.0, 647.0, 709.0, 1194.0, 868.0, 530.0, 197.0, 493.0, 896.0, 1536.0, 100.0, 1636.0, 1293.0, 1650.0, 1169.0, 1651.0, 529.0, 1664.0, 30.0, 1645.0, 1452.0, 1458.0, 57.0, 1142.0, 190.0, 135.0, 514.0, 183.0, 1080.0, 657.0, 318.0, 192.0, 1500.0, 484.0, 528.0, 169.0, 1623.0, 1398.0, 89.0, 519.0, 656.0, 511.0, 513.0, 302.0, 59.0, 1396.0, 1463.0, 1642.0, 474.0, 1639.0, 179.0, 199.0, 48.0, 1064.0, 603.0, 178.0, 344.0, 427.0, 488.0, 408.0, 1524.0, 357.0, 1449.0, 23.0, 480.0, 98.0, 114.0, 641.0, 479.0, 134.0, 661.0, 127.0, 285.0, 483.0]}
```